# TipTop — Papyrus Tutorial

*Last update: October, 2025*

**Objective:** Build a TipTop parameter file for the *Papyrus* system, verify key optical/AO quantities, and run PSF simulations.

> This notebook is organized in short steps with **TODO** cells for you to complete.

## Prerequisites

- Python 3.11+ recommended
- Packages: `tiptop v.1.3.29`, `astropy`, `matplotlib`, `numpy`
- Basic knowledge: r₀, seeing, λ/D, SR, FWHM.
- **Papyrus base INI file** (starter parameter set) — download [here](https://astro-tiptop-services.github.io/astro-tiptop-services/resources/AO_school)


## Notebook structure
1. Overview of required parameter file sections.
2. Create a `papyrus.ini` skeleton 👉 Complete the TODOs.
3. Quick exercise: diffraction-limited resolution.
4. Run TipTop with Papyrus parameters.
5. Error Budget Mini-Exercises

---


## 1) Minimum Required Sections

Regardless of the AO architecture, every configuration starts with the **telescope** and the **atmosphere**.

### Always required
- **`[telescope]`**  
  - `TelescopeDiameter` *(m)*
  - Optional: `Resolution` *(pixels across the pupil)*, `ObscurationRatio`, `ZenithAngle`
- **`[atmosphere]`**  
  - Provide **either** `Seeing` *(arcsec)* **or** `r0_Value` (m)  
  - Optional: `Wavelength` (reference λ for the above, default _@500nm_), `L0`, `Cn2Weights`, `Cn2Heights`, `WindSpeed`,`WindDirection`

- **`[sources_science]`** *(your science target)*  
  - `Wavelength` (scalar or list in meters)  
  - Optional: `Zenith`, `Azimuth` (default 0 if omitted)

- **`[sensor_science]`** *(image formation / detector sampling)*  
  - `PixelScale` *(mas/pixel)*  
  - `FieldOfView` *(pixel/spaxel)*

### AO modules (conditional)
**High-Order (HO) with a Natural Guide Star (NGS) — SCAO**  
  If you use a single NGS for wavefront sensing:
  - **`[sources_HO]`** (the guide star properties; typically wavelength/position)
  - **`[sensor_HO]`** (the WFS configuration; e.g., `WfsType='pyramid'`, `NumberPhotons`, `SigmaRON`, etc.)
  - **`[DM]`** (deformable mirror geometry; e.g., `NumberActuators`, `DmPitchs`, `DmHeights`)
  - **`[RTC]`** (control loop; e.g., `LoopGain_HO`, `SensorFrameRate_HO`, `LoopDelaySteps_HO`)

> **Rule of thumb:** if you define any `sources_*` block for sensing, you **must** also define the matching `sensor_*` block (and vice-versa).

---

### Minimal SCAO (NGS) checklist
- `[telescope]` with `TelescopeDiameter` 
- `[atmosphere]` with **either** `Seeing` **or** `r0_Value` 
- `[sources_science]` with your science `Wavelength`(s)  
- `[sensor_science]` with `PixelScale`  
- `[sources_HO]` **and** `[sensor_HO]` (NGS + WFS settings)  
- `[DM]` (actuator layout)  
- `[RTC]` (loop gain, frame rate, delay)

---


## 2) Skeleton `papyrus.ini` file
👉 [Download it](https://astro-tiptop-services.github.io/astro-tiptop-services/resources/AO_school) and Open it and fill the `TODO`.

💡**Need help?**  
- TIPTOP How to set up: https://astro-tiptop-services.github.io/astro-tiptop-services/docs/orion/howtosetup  
- Parameter files reference: https://astro-tiptop-services.github.io/astro-tiptop-services/docs/orion/parameterfiles

## 3) Exercise — Diffraction-limited Resolution 

Compute the diffraction-limited FWHM (approx. in arcseconds) for the science object.

_Indicative formula for an unobstructed circular pupil: `θ≈1.03λ/D` (in radians) → convert to arcseconds._

Given your detector pixel scale (mas/pixel), does it satisfy the Nyquist criterion at both wavelengths? <br/>
_(Hint: Nyquist requires ≥2 pixels per FWHM; i.e. pixel scale ≤ FWHM/2)._

In [ ]:

import numpy as np

# Set your telescope diameter in meters
D = TODO # 👈

# Set the Science Wavelength in meters
lambda_src = TODO  # 👈

K = 1.03 * 206265  # rad -> arcsec 

theta_src_arcsec = K * lambda_src / D

print(f"Diffraction-limited FWHM:  {theta_src_arcsec:.6f} arcsec ({theta_src_arcsec*1e3:.2f} mas)")


## 4) Run TipTop with PAPYRUS parameters




### A) Imports

In [ ]:
from tiptop.tiptop import *
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.patches import Circle
import numpy as np

### B) Define paths and filenames

- `path_in` : folder containing your `.ini` parameter file
- `path_out` : output folder (must exist or be creatable)
- `file_in` : base name (without extension) of the `.ini` file
- `file_out` : prefix for output files

In [ ]:
# %% Path configuration 
path_in = './'           # 👈 adapt to your setup
path_out = './'          # 👈 adapt to your setup
file_in = "papyrus"      # without .ini extension
file_out = "papyrus"     # output prefix

# Create output folder if needed
os.makedirs(path_out, exist_ok=True)

print("path_in :", path_in)
print("path_out:", path_out)
print("file_in :", file_in)
print("file_out:", file_out)

### C) Run TIPTOP simulation

`tiptop.overallSimulation` runs a complete TipTop simulation based on an input parameter file. The function accepts several optional arguments to enable or disable specific features and select desired outputs.
Click [here](https://astro-tiptop-services.github.io/astro-tiptop-services/docs/orion/howtosetuplaunchfile#overallSimulation) to access to the overallSimulation function documentation.

In [ ]:
overallSimulation(path_in, file_in, path_out, file_out, verbose=True, getHoErrorBreakDown=True)
print("✅ Simulation complete")

### D) Open the output FITS and extract data

We extract:
- **AO-corrected PSF** (HDU 1)
- **Seeing-limited PSF** (HDU 2)
- **Diffraction-limited PSF** (HDU 3)
- **Radial profiles** (HDU 4 or 5 depending on save options)
- Useful metadata from the FITS header (wavelength, pixel scale, Strehl, FWHM)

Click [here](https://astro-tiptop-services.github.io/astro-tiptop-services/docs/orion/howtosetuplaunchfile#simulation_output) to the simulation output documentation.


In [ ]:
# %% Open FITS file
fits_path = os.path.join(path_out, file_out + ".fits")
print("Reading:", fits_path)

ao = aoSystem(f"{path_in}{file_in}.ini")

with fits.open(fits_path) as hdul:
    hdul.info()

    psf_ao   = hdul[1].data[0, ...]  # AO-corrected
    psf_turb = hdul[2].data          # seeing-limited
    psf_dl   = hdul[3].data          # diffraction-limited

    # Profiles (HDU 4 or 5 depending on CONTENT)
    header4 = hdul[4].header
    if header4.get('CONTENT') == 'Final PSFs profiles':
        profiles = hdul[4].data
    else:
        profiles = hdul[5].data

    # Metadata
    psf_header = hdul[1].header
    wvl = float(psf_header.get("WL_NM"))        # wavelength (nm)
    pix_mas = float(psf_header.get("PIX_MAS"))  # pixel scale (mas)

    # ✅ corrected from original script:
    sr_key   = f"SR{0:04d}"
    fwhm_key = f"FWHM{0:04d}"
    sr   = psf_header.get(sr_key)    # Strehl ratio
    fwhm = psf_header.get(fwhm_key)  # FWHM (mas)
    hdul.close()

print("Pixel scale [mas]:", pix_mas)
print("Strehl Ratio:", sr)
print("FWHM [mas]:", fwhm)
print("Wvl [nm]:", wvl)

### E) Normalize PSFs and build axis in arcseconds

In [6]:
# %% Normalize
psf_ao   = psf_ao / np.sum(psf_ao)
psf_dl   = psf_dl / np.sum(psf_dl)
psf_turb = psf_turb / np.sum(psf_turb)

# Axis in arcsec
nx = psf_ao.shape[0]
axis = np.linspace(-nx//2, nx//2, nx) * pix_mas * 1e-3  # mas -> arcsec

crop_size = 4
nx = FourierUtils.cropSupport(psf_ao, crop_size).shape[0]
axis = np.linspace(-nx//2, nx//2, nx) * pix_mas * 1e-3

### F) Visualize PSFs (log scale)

We adjust the dynamic range to display down to $10^{-5}$ of the maximum.


In [7]:
# %% Plot PSFs
def plot_psf(psf, title, position):
    plt.subplot(1, 3, position)
    im = plt.imshow(psf, norm=norm, cmap='Spectral_r',
                    extent=[axis[0], axis[-1], axis[0], axis[-1]])
    plt.title(title, pad=10)
    plt.xlabel('[arcsec]')
    if position == 1:
        plt.ylabel('[arcsec]')
    else:
        plt.ylabel('')
    plt.colorbar(im, fraction=0.046)

In [ ]:
psf_max = psf_ao.max()
vmax = psf_max
vmin = psf_max * 1e-4
norm = LogNorm(vmin=vmin, vmax=vmax)
rad2arcsec = 206265
control_radius = ao.src.wvl/ao.tel.D*ao.dms.nActu1D[0]/2*rad2arcsec

plt.figure(1, figsize=(20, 5))
plt.suptitle(rf'$\lambda_{{\mathrm{{science}}}} = {int(wvl)}$ nm', y=1)
plt.subplots_adjust(top=0.85)
plot_psf(FourierUtils.cropSupport(psf_ao, crop_size),  f'AO (SR={sr*100:.1f}%, FWHM={fwhm:.1f} mas)', 1)
corr_zone = Circle([0,0], control_radius, fc='none', ec='k', ls=':')
plt.gca().add_artist(corr_zone)
plot_psf(FourierUtils.cropSupport(psf_dl, crop_size),  'Diffraction', 2)
plot_psf(FourierUtils.cropSupport(psf_turb, crop_size),'Open loop', 3)
plt.show()

### G) Radial Profile (AO PSF) 

In [9]:
# Radial profile
radii = profiles[0, 0, :]*1e-3
p_norm = profiles[1, 0, :] / np.max(profiles[1, 0, :])

In [ ]:
# Plots
plt.figure(figsize=(8, 6))
plt.title('PSF')
plt.semilogy(radii, p_norm, label='AO profile')
plt.axvline(control_radius, c='k', ls=':', label='Control radius AO')
plt.xlabel('Radial distance (arcsec)')
plt.ylabel('PSF profile (normalized)')
plt.title(f'Radial profile - AO PSF - @{int(wvl)} nm')
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray', alpha=0.7)

plt.show()

## 5) Error Budget Mini-Exercises 

These exercises explore the main contributions to the Adaptive Optics **error budget**: 
- Temporal error (servo-lag)  
- Measurement noise (photon flux / WFS RON)  
- Fitting error (DM actuator sampling)  
- Atmospheric effects (seeing, wind speed)

**Goal.** Explore how key parameters impact AO error terms.

---

### Baseline
Start from your **Papyrus baseline configuration**:  
- Same seeing (r0), wind speed, telescope diameter, pixel scale, DM geometry.  
- Keep everything fixed **except** the parameter under test.

---

### 1) **Temporal error**  

You will investigate how timing limitations in the control loop impact the overall AO performance.

**Goal.** Identify a system parameter linked to the loop time response and explore how changing it affects:
- The **spatio-temporal error** in TipTop’s error breakdown,
- The PSF shape and any metrics you consider relevant.

>  ✍️ *What general behaviour do you expect when temporal response improves or degrades?*

In [ ]:
#TODO

### 2) **Measurement noise** 

Identify a parameter that influences the signal quality in the wavefront sensor, and explore how changing it affects:
- the noise-related contribution in the error breakdown,
- the PSF shape and any metrics you consider relevant.

> ✍️ *How would you expect measurement noise and performance to vary as signal conditions improve or deteriorate?*

In [ ]:
#TODO

### 3) **Fitting error** 

**Goal.** Identify a parameter that controls how finely the deformable mirror can approximate the incoming wavefront, and explore how changing it affects:
- the fitting-related contribution in the error breakdown,
- the PSF shape and any metrics you consider relevant.

> ✍️ *How would you expect the ability to correct higher spatial frequencies to influence the fitting error?*

In [ ]:
#TODO

### 4) **Atmospheric effects** (seeing and wind)

**Goal.** Identify one (or more) atmospheric parameter(s) that influence the turbulence strength or its temporal evolution, and explore how modifying them affects:
- the PSF shape and any metrics you consider relevant,
- the related contributions in the error breakdown.

> ✍️ *In what ways do changing atmospheric conditions impact the AO correction and the dfferent error terms?*

In [ ]:
#TODO

## 🎉 That's it… or is it?

You've reached the end of the tutorial !  
At this point, TipTop won’t stop you from going further… 

Check out our official website to discover more about all of TipTop’s features, and don’t hesitate to provide feedback in the future!

🔭 Have fun, and may your PSFs stay sharp!